In [422]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import sqlite3
import re
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [423]:
basics_ratings = pd.read_csv('../Data/movie_basics_rating.csv')
tmdb = pd.read_csv('../Data/tmdb_df.csv')
budget = pd.read_csv('../Data/movie_budget.csv')
info = pd.read_csv('../Data/movie_info.csv')
gross = pd.read_csv('../Data/bom_movie_gross.csv')

In [424]:
basics_ratings

,Unnamed: 0,movie_id,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes
0,0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77
1,1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama",7.2,43
2,2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama,6.9,4517
3,3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama",6.1,13
4,4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy",6.5,119
...,...,...,...,...,...,...,...,...,...
73851,73851,tt9913084,Diabolik sono io,Diabolik sono io,2019,75.0,Documentary,6.2,6
73852,73852,tt9914286,Sokagin Çocuklari,Sokagin Çocuklari,2019,98.0,"Drama,Family",8.7,136
73853,73853,tt9914642,Albatross,Albatross,2017,NaN,Documentary,8.5,8
73854,73854,tt9914942,La vida sense la Sara Amat,La vida sense la Sara Amat,2019,NaN,NaN,6.6,5


In [425]:
tmdb

,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186
...,...,...,...,...,...,...,...,...,...,...
25492,26512,"[27, 18]",488143,en,Laboratory Conditions,0.600,2018-10-13,Laboratory Conditions,0.0,1
25493,26513,"[18, 53]",485975,en,_EXHIBIT_84xxx_,0.600,2018-05-01,_EXHIBIT_84xxx_,0.0,1
25494,26514,"[14, 28, 12]",381231,en,The Last One,0.600,2018-10-01,The Last One,0.0,1
25495,26515,"[10751, 12, 28]",366854,en,Trailer Made,0.600,2018-06-22,Trailer Made,0.0,1


In [426]:
type(tmdb['genre_ids'])

pandas.core.series.Series

In [427]:
genre_map = {28: 'Action', 12: 'Adventure', 16: 'Animation', 35: 'Comedy',
             80: 'Crime', 99: 'Documentary', 18: 'Drama', 10751: 'Family', 
             14: 'Fantasy', 36: 'History', 27: 'Horror', 10402: 'Music', 
             9648: 'Mystery', 10749: 'Romance', 878: 'Science Fiction', 
             10770: 'TV Movie', 53: 'Thriller', 10752: 'War', 37: 'Western'}
#tmdb['genres'] = tmdb['genre_ids'].apply(lambda x: [genre_map[int(genre_id)] for genre_id in eval(x)])
tmdb['genres_labels'] = tmdb['genre_ids'].apply(lambda x: ', '.join([genre_map[int(genre_id)] for genre_id in eval(x)]))
tmdb

,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count,genres_labels
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788,"Adventure, Fantasy, Family"
1,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610,"Fantasy, Adventure, Animation, Family"
2,2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368,"Adventure, Action, Science Fiction"
3,3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174,"Animation, Comedy, Family"
4,4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186,"Action, Science Fiction, Adventure"
...,...,...,...,...,...,...,...,...,...,...,...
25492,26512,"[27, 18]",488143,en,Laboratory Conditions,0.600,2018-10-13,Laboratory Conditions,0.0,1,"Horror, Drama"
25493,26513,"[18, 53]",485975,en,_EXHIBIT_84xxx_,0.600,2018-05-01,_EXHIBIT_84xxx_,0.0,1,"Drama, Thriller"
25494,26514,"[14, 28, 12]",381231,en,The Last One,0.600,2018-10-01,The Last One,0.0,1,"Fantasy, Action, Adventure"
25495,26515,"[10751, 12, 28]",366854,en,Trailer Made,0.600,2018-06-22,Trailer Made,0.0,1,"Family, Adventure, Action"


In [428]:
budget

,Unnamed: 0,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,net_profit
0,0,1,"Dec 18, 2009",Avatar,425000000.0,760507625.0,2.776345e+09,2.351345e+09
1,1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000.0,241063875.0,1.045664e+09,6.350639e+08
2,2,3,"Jun 7, 2019",Dark Phoenix,350000000.0,42762350.0,1.497624e+08,-2.002376e+08
3,3,4,"May 1, 2015",Avengers: Age of Ultron,330600000.0,459005868.0,1.403014e+09,1.072414e+09
4,4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000.0,620181382.0,1.316722e+09,9.997217e+08
...,...,...,...,...,...,...,...,...
5777,5777,78,"Dec 31, 2018",Red 11,7000.0,0.0,0.000000e+00,-7.000000e+03
5778,5778,79,"Apr 2, 1999",Following,6000.0,48482.0,2.404950e+05,2.344950e+05
5779,5779,80,"Jul 13, 2005",Return to the Land of Wonders,5000.0,1338.0,1.338000e+03,-3.662000e+03
5780,5780,81,"Sep 29, 2015",A Plague So Pleasant,1400.0,0.0,0.000000e+00,-1.400000e+03


In [429]:
info

,Unnamed: 0,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,runtime,studio
0,0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",104 minutes,NaN
1,1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",108 minutes,Entertainment One
2,2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",116 minutes,NaN
3,3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",128 minutes,NaN
4,4,7,NaN,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaN,NaN,200 minutes,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1555,1555,1996,Forget terrorists or hijackers -- there's a ha...,R,Action and Adventure|Horror|Mystery and Suspense,NaN,NaN,"Aug 18, 2006","Jan 2, 2007",106 minutes,New Line Cinema
1556,1556,1997,The popular Saturday Night Live sketch was exp...,PG,Comedy|Science Fiction and Fantasy,Steve Barron,Terry Turner|Tom Davis|Dan Aykroyd|Bonnie Turner,"Jul 23, 1993","Apr 17, 2001",88 minutes,Paramount Vantage
1557,1557,1998,"Based on a novel by Richard Powell, when the l...",G,Classics|Comedy|Drama|Musical and Performing Arts,Gordon Douglas,NaN,"Jan 1, 1962","May 11, 2004",111 minutes,NaN
1558,1558,1999,The Sandlot is a coming-of-age story about a g...,PG,Comedy|Drama|Kids and Family|Sports and Fitness,David Mickey Evans,David Mickey Evans|Robert Gunter,"Apr 1, 1993","Jan 29, 2002",101 minutes,NaN


In [430]:
gross

,Unnamed: 0,title,studio,domestic_gross,foreign_gross,year
0,0,Toy Story 3,BV,415000000.0,652000000.0,2010
1,1,Alice in Wonderland (2010),BV,334200000.0,691300000.0,2010
2,2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000.0,2010
3,3,Inception,WB,292600000.0,535700000.0,2010
4,4,Shrek Forever After,P/DW,238700000.0,513900000.0,2010
...,...,...,...,...,...,...
3382,3382,The Quake,Magn.,6200.0,NaN,2018
3383,3383,Edward II (2018 re-release),FM,4800.0,NaN,2018
3384,3384,El Pacto,Sony,2500.0,NaN,2018
3385,3385,The Swan,Synergetic,2400.0,NaN,2018


- merge basics_ratings & tmdb on original_title = basics_ratings_tmdb
- merge budget & gross on movie & title, respectively = budget_gross
- basics_ratings_tmdb & budget_gross on...
- info = irrelevant?

In [431]:
#merging basics_ratings & tmdb
basics_ratings_tmdb = pd.merge(basics_ratings, tmdb, on='original_title', how='left')
basics_ratings_tmdb

,Unnamed: 0_x,movie_id,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes,Unnamed: 0_y,genre_ids,id,original_language,popularity,release_date,title,vote_average,vote_count,genres_labels
0,0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama",7.2,43,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama,6.9,4517,24185.0,"[35, 18]",299782.0,en,9.800,2018-11-02,The Other Side of the Wind,7.0,64.0,"Comedy, Drama"
3,3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama",6.1,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy",6.5,119,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75280,73852,tt9914286,Sokagin Çocuklari,Sokagin Çocuklari,2019,98.0,"Drama,Family",8.7,136,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75281,73853,tt9914642,Albatross,Albatross,2017,NaN,Documentary,8.5,8,5611.0,[18],72842.0,en,5.624,2012-01-02,Albatross,6.2,48.0,Drama
75282,73853,tt9914642,Albatross,Albatross,2017,NaN,Documentary,8.5,8,23077.0,[99],532558.0,en,0.600,2017-01-01,Albatross,10.0,1.0,Documentary
75283,73854,tt9914942,La vida sense la Sara Amat,La vida sense la Sara Amat,2019,NaN,NaN,6.6,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [432]:
basics_ratings_tmdb.drop(['Unnamed: 0_x'], axis=1)

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes,Unnamed: 0_y,genre_ids,id,original_language,popularity,release_date,title,vote_average,vote_count,genres_labels
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama",7.2,43,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama,6.9,4517,24185.0,"[35, 18]",299782.0,en,9.800,2018-11-02,The Other Side of the Wind,7.0,64.0,"Comedy, Drama"
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama",6.1,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy",6.5,119,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75280,tt9914286,Sokagin Çocuklari,Sokagin Çocuklari,2019,98.0,"Drama,Family",8.7,136,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75281,tt9914642,Albatross,Albatross,2017,NaN,Documentary,8.5,8,5611.0,[18],72842.0,en,5.624,2012-01-02,Albatross,6.2,48.0,Drama
75282,tt9914642,Albatross,Albatross,2017,NaN,Documentary,8.5,8,23077.0,[99],532558.0,en,0.600,2017-01-01,Albatross,10.0,1.0,Documentary
75283,tt9914942,La vida sense la Sara Amat,La vida sense la Sara Amat,2019,NaN,NaN,6.6,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [433]:
budget_gross = pd.merge(budget,
                       gross,
                       left_on='movie',
                       right_on='title',
                       how='left')
budget_gross

,Unnamed: 0_x,id,release_date,movie,production_budget,domestic_gross_x,worldwide_gross,net_profit,Unnamed: 0_y,title,studio,domestic_gross_y,foreign_gross,year
0,0,1,"Dec 18, 2009",Avatar,425000000.0,760507625.0,2.776345e+09,2.351345e+09,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000.0,241063875.0,1.045664e+09,6.350639e+08,330.0,Pirates of the Caribbean: On Stranger Tides,BV,241100000.0,804600000.0,2011.0
2,2,3,"Jun 7, 2019",Dark Phoenix,350000000.0,42762350.0,1.497624e+08,-2.002376e+08,NaN,NaN,NaN,NaN,NaN,NaN
3,3,4,"May 1, 2015",Avengers: Age of Ultron,330600000.0,459005868.0,1.403014e+09,1.072414e+09,1875.0,Avengers: Age of Ultron,BV,459000000.0,946400000.0,2015.0
4,4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000.0,620181382.0,1.316722e+09,9.997217e+08,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5777,5777,78,"Dec 31, 2018",Red 11,7000.0,0.0,0.000000e+00,-7.000000e+03,NaN,NaN,NaN,NaN,NaN,NaN
5778,5778,79,"Apr 2, 1999",Following,6000.0,48482.0,2.404950e+05,2.344950e+05,NaN,NaN,NaN,NaN,NaN,NaN
5779,5779,80,"Jul 13, 2005",Return to the Land of Wonders,5000.0,1338.0,1.338000e+03,-3.662000e+03,NaN,NaN,NaN,NaN,NaN,NaN
5780,5780,81,"Sep 29, 2015",A Plague So Pleasant,1400.0,0.0,0.000000e+00,-1.400000e+03,NaN,NaN,NaN,NaN,NaN,NaN


In [434]:
df = pd.merge(basics_ratings_tmdb, 
                  budget_gross,
                  left_on='primary_title',
                  right_on='movie',
                  how='left')
df

,Unnamed: 0_x_x,movie_id,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes,Unnamed: 0_y_x,...,production_budget,domestic_gross_x,worldwide_gross,net_profit,Unnamed: 0_y_y,title_y,studio,domestic_gross_y,foreign_gross,year
0,0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama",7.2,43,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama,6.9,4517,24185.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama",6.1,13,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy",6.5,119,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75485,73852,tt9914286,Sokagin Çocuklari,Sokagin Çocuklari,2019,98.0,"Drama,Family",8.7,136,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75486,73853,tt9914642,Albatross,Albatross,2017,NaN,Documentary,8.5,8,5611.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75487,73853,tt9914642,Albatross,Albatross,2017,NaN,Documentary,8.5,8,23077.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75488,73854,tt9914942,La vida sense la Sara Amat,La vida sense la Sara Amat,2019,NaN,NaN,6.6,5,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [435]:
df.isna().sum()

Unnamed: 0_x_x           0
movie_id                 0
primary_title            0
original_title           0
start_year               0
runtime_minutes       7717
genres                 816
averagerating            0
numvotes                 0
Unnamed: 0_y_x       58390
genre_ids            58390
id_x                 58390
original_language    58390
popularity           58390
release_date_x       58390
title_x              58390
vote_average         58390
vote_count           58390
genres_labels        58390
Unnamed: 0_x_y       72157
id_y                 72157
release_date_y       72157
movie                72157
production_budget    72157
domestic_gross_x     72157
worldwide_gross      72157
net_profit           72157
Unnamed: 0_y_y       73912
title_y              73912
studio               73912
domestic_gross_y     73913
foreign_gross        74154
year                 73912
dtype: int64

In [436]:
df=df.drop(['Unnamed: 0_x_x', 'Unnamed: 0_y_x', 'original_language'], axis=1)
df

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes,genre_ids,id_x,...,production_budget,domestic_gross_x,worldwide_gross,net_profit,Unnamed: 0_y_y,title_y,studio,domestic_gross_y,foreign_gross,year
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama",7.2,43,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama,6.9,4517,"[35, 18]",299782.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama",6.1,13,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy",6.5,119,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75485,tt9914286,Sokagin Çocuklari,Sokagin Çocuklari,2019,98.0,"Drama,Family",8.7,136,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75486,tt9914642,Albatross,Albatross,2017,NaN,Documentary,8.5,8,[18],72842.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75487,tt9914642,Albatross,Albatross,2017,NaN,Documentary,8.5,8,[99],532558.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75488,tt9914942,La vida sense la Sara Amat,La vida sense la Sara Amat,2019,NaN,NaN,6.6,5,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [437]:
df_profit_genres=df.loc(axis=1)['primary_title', 'genres', 'genres_labels', 'net_profit']
df_profit_genres
# relevant columns to analyse genres & profits

,primary_title,genres,genres_labels,net_profit
0,Sunghursh,"Action,Crime,Drama",NaN,NaN
1,One Day Before the Rainy Season,"Biography,Drama",NaN,NaN
2,The Other Side of the Wind,Drama,"Comedy, Drama",NaN
3,Sabse Bada Sukh,"Comedy,Drama",NaN,NaN
4,The Wandering Soap Opera,"Comedy,Drama,Fantasy",NaN,NaN
...,...,...,...,...
75485,Sokagin Çocuklari,"Drama,Family",NaN,NaN
75486,Albatross,Documentary,Drama,NaN
75487,Albatross,Documentary,Documentary,NaN
75488,La vida sense la Sara Amat,NaN,NaN,NaN


In [438]:
#df_profit_genres=df_profit_genres.replace({28: 'Action', 12: 'Adventure', 16: 'Animation', 35: 'Comedy', 
#                                       80: 'Crime', 99: 'Documentary', 18: 'Drama', 10751: 'Family', 
#                                       14: 'Fantasy', 36: 'History', 27: 'Horror', 10402: 'Music', 
#                                      9648: 'Mystery', 10749: 'Romance', 878: 'Science Fiction', 
#                                      10770: 'TV Movie', 53: 'Thriller', 10752: 'War', 37: 'Western'}, inplace = True)
#df_profit_genres

In [439]:
df_profit_genres.isna().sum()

primary_title        0
genres             816
genres_labels    58390
net_profit       72157
dtype: int64

In [440]:
df_profit_genres=df_profit_genres.dropna()
df_profit_genres

,primary_title,genres,genres_labels,net_profit
16,Foodfight!,"Action,Animation,Comedy","Animation, Action, Comedy, Family",-4.492629e+07
41,On the Road,"Adventure,Drama,Romance","Adventure, Drama",-1.568670e+07
47,The Secret Life of Walter Mitty,"Adventure,Comedy,Drama","Adventure, Comedy, Drama, Fantasy",9.686118e+07
51,A Walk Among the Tombstones,"Action,Crime,Drama","Crime, Drama, Mystery, Thriller",3.410859e+07
52,Jurassic World,"Action,Adventure,Sci-Fi","Action, Adventure, Science Fiction, Thriller",1.433855e+09
...,...,...,...,...
75213,Walter,Comedy,"Drama, Comedy",-7.000000e+05
75407,Sisters,"Action,Drama",Comedy,7.603066e+07
75464,Columbus,Comedy,Drama,4.105110e+05
75477,Unstoppable,Documentary,"Action, Thriller",7.072092e+07


In [441]:
df_profit_genres.duplicated()

16       False
41       False
47       False
51       False
52       False
         ...  
75213    False
75407    False
75464     True
75477     True
75478     True
Length: 2690, dtype: bool

In [442]:
df_profit_genres.drop_duplicates()

,primary_title,genres,genres_labels,net_profit
16,Foodfight!,"Action,Animation,Comedy","Animation, Action, Comedy, Family",-4.492629e+07
41,On the Road,"Adventure,Drama,Romance","Adventure, Drama",-1.568670e+07
47,The Secret Life of Walter Mitty,"Adventure,Comedy,Drama","Adventure, Comedy, Drama, Fantasy",9.686118e+07
51,A Walk Among the Tombstones,"Action,Crime,Drama","Crime, Drama, Mystery, Thriller",3.410859e+07
52,Jurassic World,"Action,Adventure,Sci-Fi","Action, Adventure, Science Fiction, Thriller",1.433855e+09
...,...,...,...,...
74882,The Gambler,"Action,Sci-Fi,Thriller","Thriller, Crime, Drama",1.371803e+07
74883,The Gambler,"Action,Sci-Fi,Thriller","Thriller, Crime, Drama",-2.898227e+06
75162,Bait,Drama,"Action, Horror, Thriller",-1.952803e+07
75213,Walter,Comedy,"Drama, Comedy",-7.000000e+05


In [443]:
type(df_profit_genres['genres'])

pandas.core.series.Series

In [445]:
type(df_profit_genres['genres_labels'])

pandas.core.series.Series

In [446]:
df_profit_genres.sort_values('net_profit', ascending=False).head(10)

,primary_title,genres,genres_labels,net_profit
44572,Avengers: Infinity War,"Action,Adventure,Sci-Fi","Adventure, Action, Fantasy",1.748134e+09
52,Jurassic World,"Action,Adventure,Sci-Fi","Action, Adventure, Science Fiction, Thriller",1.433855e+09
341,The Avengers,"Action,Adventure,Sci-Fi","Science Fiction, Action, Adventure",1.292936e+09
11582,Black Panther,"Action,Adventure,Sci-Fi","Action, Adventure, Fantasy, Science Fiction",1.148258e+09
11581,Black Panther,"Action,Adventure,Sci-Fi","Action, Animation",1.148258e+09
50685,Jurassic World: Fallen Kingdom,"Action,Adventure,Sci-Fi","Action, Adventure, Science Fiction",1.135773e+09
22184,Frozen,"Adventure,Animation,Comedy","Animation, Adventure, Family",1.122470e+09
2117,Frozen,"Adventure,Drama,Sport","Animation, Adventure, Family",1.122470e+09
22183,Frozen,"Adventure,Animation,Comedy",Thriller,1.122470e+09
2116,Frozen,"Adventure,Drama,Sport",Thriller,1.122470e+09


In [447]:
df_profit_genres.shape

(2690, 4)

In [448]:
df_profit_genres['genres'].nunique()


279

In [449]:
df_profit_genres['genres'].unique()

array(['Action,Animation,Comedy', 'Adventure,Drama,Romance',
       'Adventure,Comedy,Drama', 'Action,Crime,Drama',
       'Action,Adventure,Sci-Fi', 'Comedy,Drama', 'Comedy,Family',
       'Comedy,Drama,Romance', 'Adventure,Animation,Comedy',
       'Action,Thriller', 'Comedy', 'Action,Adventure,Thriller',
       'Horror,Mystery,Thriller', 'Action,Drama,Family',
       'Drama,Romance,Sci-Fi', 'Biography,Drama,History',
       'Action,Adventure,Animation', 'Action,Adventure,Fantasy',
       'Biography,Drama,Sport', 'Adventure,Drama,Fantasy',
       'Horror,Thriller', 'Action,Crime,Thriller', 'Crime,Thriller',
       'Comedy,Horror', 'Drama', 'Drama,Mystery,Sci-Fi',
       'Action,Comedy,Crime', 'Comedy,Drama,Music', 'Drama,Fantasy',
       'Action,Adventure,Comedy', 'Action,Adventure,Crime',
       'Comedy,Romance', 'Action,Adventure,Drama',
       'Adventure,Drama,History', 'Drama,History,Romance',
       'Adventure,Comedy,Family', 'Drama,Horror,Mystery',
       'Documentary,Music', '

In [451]:
df_profit_genres['genres_labels'].nunique()

620

In [452]:
df_profit_genres['genres_labels'].unique()

array(['Animation, Action, Comedy, Family', 'Adventure, Drama',
       'Adventure, Comedy, Drama, Fantasy',
       'Crime, Drama, Mystery, Thriller',
       'Action, Adventure, Science Fiction, Thriller', 'Drama, Comedy',
       'Comedy', 'Romance, Comedy, Drama', 'Animation, Family',
       'Action, Adventure, Science Fiction',
       'Action, Adventure, Comedy, Crime, Drama',
       'Thriller, Action, Adventure, Comedy, Crime', 'Horror, Thriller',
       'Action, Science Fiction, Drama', 'Drama, Science Fiction',
       'Animation, Family, Comedy', 'History, Drama',
       'Drama, Romance, Comedy',
       'Action, Adventure, Animation, Family, Fantasy, Comedy',
       'Action, Fantasy', 'Adventure, Fantasy, War, Romance', 'Drama',
       'Adventure, Drama, Action', 'Mystery, Horror, Action',
       'Thriller, Action, Crime', 'Crime, Thriller', 'Thriller, Horror',
       'Comedy, Horror', 'Action, Comedy',
       'Animation, Family, Adventure, Comedy, Fantasy',
       'Action, Thrille

- merge 'genres' & 'genres_labels' columns, remove duplicate genres?